<a href="https://colab.research.google.com/github/JSJeong-me/GPT-Graph/blob/main/01-Intro-LangGraph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### https://python.langchain.com/docs/langgraph

In [ ]:
!pip install langgraph

In [ ]:
!pip install -U langchain langchain_openai tavily-python

In [ ]:
!pip install python-dotenv

In [8]:
!echo "OPENAI_API_KEY=sk-b" >> .env
!echo "TAVILY_API_KEY=tvly-" >> .env
!echo "LANGCHAIN_API_KEY=ls__" >> .env
!source /content/.env

import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()
# Access the API key using the variable name defined in the .env file
api_key = os.getenv("OPENAI_API_KEY")
api_key = os.getenv("TAVILY_API_KEY")
api_key = os.getenv("LANGCHAIN_API_KEY")

In [9]:
from langchain_community.tools.tavily_search import TavilySearchResults

tools = [TavilySearchResults(max_results=1)]

In [10]:
from langgraph.prebuilt import ToolExecutor

tool_executor = ToolExecutor(tools)

In [11]:
from langchain_openai import ChatOpenAI

# We will set streaming=True so that we can stream tokens
# See the streaming section for more information on this.
model = ChatOpenAI(temperature=0, streaming=True)  # model="gpt-3.5-turbo-1106", streaming=True

### 모델이 호출할 수 있는 도구가 있다는 것을 모델이 알고 있는지 확인해야 합니다. LangChain 도구를 OpenAI 함수 호출을 위한 형식으로 변환한 다음 이를 모델 클래스에 바인딩하면 됩니다.

In [ ]:
from langchain.tools.render import format_tool_to_openai_function

functions = [format_tool_to_openai_function(t) for t in tools]
model = model.bind_functions(functions)

### Define the agent state

In [13]:
from typing import TypedDict, Annotated, Sequence
import operator
from langchain_core.messages import BaseMessage


class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]

### Define the nodes

In langgraph, a node can be either a function or a runnable. There are two main nodes we need for this:

The agent: responsible for deciding what (if any) actions to take.
A function to invoke tools: if the agent decides to take an action, this node will then execute that action.

Conditional Edge: after the agent is called, we should either:

a. If the agent said to take an action, then the function to invoke tools should be called

b. If the agent said that it was finished, then it should finish

Normal Edge: after the tools are invoked, it should always go back to the agent to decide what to do next

In [14]:
from langgraph.prebuilt import ToolInvocation
import json
from langchain_core.messages import FunctionMessage

# Define the function that determines whether to continue or not
def should_continue(state):
    messages = state['messages']
    last_message = messages[-1]
    # If there is no function call, then we finish
    if "function_call" not in last_message.additional_kwargs:
        return "end"
    # Otherwise if there is, we continue
    else:
        return "continue"

# Define the function that calls the model
def call_model(state):
    messages = state['messages']
    response = model.invoke(messages)
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}

# Define the function to execute tools
def call_tool(state):
    messages = state['messages']
    # Based on the continue condition
    # we know the last message involves a function call
    last_message = messages[-1]
    # We construct an ToolInvocation from the function_call
    action = ToolInvocation(
        tool=last_message.additional_kwargs["function_call"]["name"],
        tool_input=json.loads(last_message.additional_kwargs["function_call"]["arguments"]),
    )
    # We call the tool_executor and get back a response
    response = tool_executor.invoke(action)
    # We use the response to create a FunctionMessage
    function_message = FunctionMessage(content=str(response), name=action.tool)
    # We return a list, because this will get added to the existing list
    return {"messages": [function_message]}

### Define the graph

In [15]:
from langgraph.graph import StateGraph, END
# Define a new graph
workflow = StateGraph(AgentState)

# Define the two nodes we will cycle between
workflow.add_node("agent", call_model)
workflow.add_node("action", call_tool)

# Set the entrypoint as `agent`
# This means that this node is the first one called
workflow.set_entry_point("agent")

# We now add a conditional edge
workflow.add_conditional_edges(
    # First, we define the start node. We use `agent`.
    # This means these are the edges taken after the `agent` node is called.
    "agent",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
    # Finally we pass in a mapping.
    # The keys are strings, and the values are other nodes.
    # END is a special node marking that the graph should finish.
    # What will happen is we will call `should_continue`, and then the output of that
    # will be matched against the keys in this mapping.
    # Based on which one it matches, that node will then be called.
    {
        # If `tools`, then we call the tool node.
        "continue": "action",
        # Otherwise we finish.
        "end": END
    }
)

# We now add a normal edge from `tools` to `agent`.
# This means that after `tools` is called, `agent` node is called next.
workflow.add_edge('action', 'agent')

# Finally, we compile it!
# This compiles it into a LangChain Runnable,
# meaning you can use it as you would any other runnable
app = workflow.compile()

### Run it!

In [16]:
from langchain_core.messages import HumanMessage

inputs = {"messages": [HumanMessage(content="what is the weather in sf")]}
app.invoke(inputs)

{'messages': [HumanMessage(content='what is the weather in sf'),
  AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "query": "weather in San Francisco"\n}', 'name': 'tavily_search_results_json'}}),
  FunctionMessage(content='[{\'url\': \'https://www.latimes.com/california/story/2024-02-04/potentially-historic-storm-brings-highest-risk-to-southern-california\', \'content\': \'Feb. 2, 2024  Jan. 31, 2024  Greater San Francisco Bay Area  Feb. 3, 2024 ‘Prepare now’: California’s next storm may bring ‘life-threatening’ flooding Feb. 2, 2024Waves crash over a breakwater in Alameda, with the San Francisco skyline in the background, on Sunday. California was facing a "potentially historic" storm raising risk of landslides, flooding...\'}]', name='tavily_search_results_json'),
  AIMessage(content="I apologize, but I couldn't find the current weather in San Francisco. However, I found an article about a potentially historic storm that is expected to bring heavy rain 

### Streaming

In [17]:
inputs = {"messages": [HumanMessage(content="what is the weather in sf")]}
for output in app.stream(inputs):
    # stream() yields dictionaries with output keyed by node name
    for key, value in output.items():
        print(f"Output from node '{key}':")
        print("---")
        print(value)
    print("\n---\n")

Output from node 'agent':
---
{'messages': [AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "query": "weather in San Francisco"\n}', 'name': 'tavily_search_results_json'}})]}

---

Output from node 'action':
---
{'messages': [FunctionMessage(content='[{\'url\': \'https://www.sfexaminer.com/news/climate/san-francisco-weather-forecast-calls-for-strongest-2024-rain/article_75347810-bfc3-11ee-abf6-e74c528e0583.html\', \'content\': "San Francisco is projected to receive 2.5 and 3 inches of rain, Clouser said, as well as gusts of wind up to 45 mph.  One of winter\'s \'stronger\' storms to douse San Francisco  Bay Area starting Wednesday at 4 a.m. and a 24-hour wind advisory in San Francisco starting at the same time.  On the heels of record-breaking heat to open the week, heavy rain is slated to pound the Bay Area on Wednesday.Temperatures will drop to a high of 60 degrees during Wednesday\'s storm and 58 Thursday. Scattered showers will linger Thursday and Frid

### Streaming LLM Tokens

In [18]:
inputs = {"messages": [HumanMessage(content="what is the weather in sf")]}
async for output in app.astream_log(inputs, include_types=["llm"]):
    # astream_log() yields the requested logs (here LLMs) in JSONPatch format
    for op in output.ops:
        if op["path"] == "/streamed_output/-":
            # this is the output from .stream()
            ...
        elif op["path"].startswith("/logs/") and op["path"].endswith(
            "/streamed_output/-"
        ):
            # because we chose to only include LLMs, these are LLM tokens
            print(op["value"])